# Secret Leakage Incident Remediation Using GitGuardian APIs

## Introduction And Format

This Jupyter Notebook project is intended to introduce you to using the [GitGuardian API](https://docs.gitguardian.com/api-docs/introduction) from the perspective of remediating an incident.  

While we cover many of the API calls available, it does not touch on all of them. For the full list, see the full [API reference documentation](https://api.gitguardian.com/docs).

A note if you are new to [Jupyter Notebooks](https://jupyter.org/): Each code block is run on demand, but are all run within the same program, sharing the same memory. This means variables set in one block are callable from each following block. If you get output that says something is undefined, make sure you ran the block that defines it. 

### Format for API calls

- The base URL for the latest version is api.gitguardian.com/v1 over HTTPS.
- All data is sent and received as JSON.
- All timestamps returned are ISO-8601 compliant, example: 2020-03-16T04:46:00+00:00 # for date-time


## 1. Authentication

In order to use the GitGuardian API, you will need to create a [new personal access token](https://docs.gitguardian.com/api-docs/personal-access-tokens). See the documentation for more details. 

Steps to create and use a Personal Access Token for this notebook:
- Go to your GitGuardian dashboard and click on API, then select the Person Access Tokens menu.
- Create a new access token with the selected scope: 
    - scan 
    - incidnet:read
    - incidnet:write
    - incidnet:share
    - members:read
- Copy the new token to the clipboard. 
- *_Do not_* paste it into this document.
- Create a new file in your project called "authorization.py" and add the following line and save the file:
    - token = 'Paste-Your-GG-API-Token-Here'
- Double check your `.gitignore` file to make sure this stays out of your source control. 

- ALTERNATIVELY - set up and use Hashicorp Vault or another such credentials manager. This is overall a better path, but let's keep things simple for this example repo.

GitGuardian relies on each request to have a header that contains a bearer token. This is the first thing we will build with the code block below. 


In [ ]:
## In this first section we authenticate
## We will need these tools throughout the examples, so let's import them all now
import requests
import authorization
import json

# This line builds our header to include with each call to the API
auth_headers = {'Authorization' : 'Token {}'.format(authorization.token) }

# Before we attempt anything fancier, let's make sure the API is working and our authorization token worked. 
response = requests.get("https://api.gitguardian.com/v1/health", headers=auth_headers)
print(response)



If you see a `<Response [200]>` above, then you are good to go!
If you see some other code, check to make sure your Access Token is valid or that your GitGuardian dashboard is available. See the full [GitGuardian API reference documentation](https://api.gitguardian.com/docs) for help with other response codes.

## List Incidents

While you can always look up the list of incidents in the dashboard and apply many different filters, you can also programmatically call the platform to get a list of incidents.  

While there are many ways to sort incidents, one convenient way to approach them is by status. 

There are [4 possible states for an incident](https://docs.gitguardian.com/internal-repositories-monitoring/remediate/prioritize-incidents#1-prioritize-with-the-incidents-table):
    - Triggered
    - Assigned
    - Ignored 
    - Resolved

Let's see if any triggered, yet-to-be-assigned incidents exist in your workspace.

To filter for just the triggered incidents, you will add `{"status":"TRIGGERED"}` in the API request, after the URL. This is true of all filters you want to apply later on. Note: It is case-sensitive.  


In [ ]:
response = requests.get(
    "http://api.gitguardian.com/v1/incidents/secrets", 
    {"status" : "TRIGGERED"}, 
    headers=auth_headers
    )
print(json.dumps(response.json(), indent=4))
print("Total retrieved triggered incidents(not assigned): "+ str(len(response.json())))
print("By default, GitGuardian only returns 20 items per page of results.")



If you have no triggered incidents, good job! That means you are on top of your secrets management game! 

But, for the purposes of this example application, we need a few to exist.  One safe way to do this is to reopen incidents that have invalid secrets and are not public.  You can do this from the GitGuardian dashboard. 

## Listing just the most recent incidents

If you have a lot of incidents, then you are likely going to want to filter this list further. 

Here is an example of filtering for:
- status=TRIGGERED - all incidents that have a status of TRIGGERED 
- ordering=-date - in an ordered list is shown descending from the most recent event
- (optional) date_after=2023-03-01 - incidents that happened after a date YYYY-MM-DD. If you have a lot of incidents this is one way you can narrow your scope. 

Later in this notebook, we will want to reference the first listed triggered event, so let's save it in a variable when we run this code block.

Here, instead of defining the filters inline with the URL, let's pre-define them for readability reasons


In [ ]:
request_filters = {
#     "date_after":"2023-03-01",
    "ordering":"-date", 
    "status":"TRIGGERED"
}

response = requests.get(
    "http://api.gitguardian.com/v1/incidents/secrets", 
    request_filters, 
    headers=auth_headers
    )

# Let's save the ID of the first listed, newest triggered incident for later.
my_incident_id = response.json()[0]['id']


## Assigning incidents

Let's assign the most recent incident to someome. In this case, you.

We will need to make our first POST command. To do that, we need to build a payload that consists of 2 things:

1. incident_id - The id of the incident to retrieve

And either one of the following:

    2. email - email of the member to assign. This parameter is mutually exclusive with member_id.
       or
    3. member_id - id of the member to assign. This parameter is mutually exclusive with email.

I trust you know your `email`. But how can we look up a workspace member's `email` or `member_id`? 

With the following API call!

If you own the GitGuardian workspace you are authenticating to, then you will be listed first. The following code saves the ID of the first user listed in the variable `my_member_id`. You can reassign this variable later, so feel free to comment that line out below. 

In [ ]:
response = requests.get(
    "https://api.gitguardian.com/v1/members", 
    headers=auth_headers
    )

print(json.dumps(response.json(), indent=4))

# Let's save the first listed member in a variable for later
my_member_id = response.json()[0]['id']


### Making the assignment call

Now you have the 
- `incident_id` 
- `member_id`, 
    - though we could use the `email` if we wanted to, 

Let's build a request to assign the first listed incident to a user using the member ID, which we found in the previous step.


In [ ]:
# If you do not want to use the variables we assigned earlier
# uncomment the appropriate line below and fill in with your pereferred IDs
# my_incident_id = "YOUR_ID"
# my_member_id = "YOUR_ID"

response = requests.post(
    "https://api.gitguardian.com/v1/incidents/secrets/{}/assign".format(my_incident_id), 
    headers=auth_headers, 
    data = {"member_id" : my_member_id}
    )

print(json.dumps(response.json(), indent=4))



## Unassign an incident

You can also unassign incidents, which puts them back to 'triggered' status if they have not yet been resolved.

The following block can be run as is since it uses the variables we set earlier. It will unassign the incident we just assigned above. 


In [ ]:
response = requests.post(
    "https://api.gitguardian.com/v1/incidents/secrets/{}/unassign".format(my_incident_id), 
    headers=auth_headers, 
    data = {"member_id" : my_member_id}
    )

# Print the updated incident data
print(json.dumps(response.json(), indent=4))


## Examining an incident

Now that you have been assigned the incident let's take a closer look at the incident data. 

When you listed the incidents, you only revealed the top layer of available data. This is equivalent to the 'Incidents view' in the dashboard. Just as in the dashboard, you need to open the view of each specific incident to reveal more details, including occurrence information; you need to drill down into the `incidents/secrets` view with the API. 


In [ ]:
response = requests.get(
    "https://api.gitguardian.com/v1/incidents/secrets/{}".format(my_incident_id), 
    headers=auth_headers
    )

print(json.dumps(response.json(), indent=4))



## Making a decision about remediating the incident

Now you have to decide:

1. Do you have enough information to resolve or ignore this incident?

or

2. Do you need to get more information from the committing author involved?

### Scenario 1: You have enough information to resolve or ignore this incident

Now that you know you can close this incident, the next decision is to `resolve` or to `ignore` the incident. 

#### Resolving incidents

Let's first resolve it. 

In order to resolve an incident, we need 2 things: 
1. `incident_id` - we already set this above as my_incident_id.
2. `secret_revoked` - this is a boolean we pass in as data; it is either true or false.


In [ ]:
response = requests.post(
    "https://api.gitguardian.com/v1/incidents/secrets/{}/resolve".format(my_incident_id), 
    headers=auth_headers, 
    data = {"secret_revoked" : "false"}
    )

print(json.dumps(response.json(), indent=4))


### Reopening incidents

You can also re-open incidents, all you need is the `incident_id`.

The following reopens the incident you just resolved. 

In [ ]:
response = requests.post(
    "https://api.gitguardian.com/v1/incidents/secrets/{}/reopen".format(my_incident_id), 
    headers=auth_headers
    )

print(json.dumps(response.json(), indent=4))


#### Ignoring Incidents

You can also choose to ignore incidents.

Yet again, we need the `incident_id`

But we also need an `ignore_reason`
This has three possible states, expressed as strings
- "test_credential"
- "false_positive"
- "low_risk"

Let's ignore this one as `low_risk`


In [ ]:
response = requests.post(
    "https://api.gitguardian.com/v1/incidents/secrets/{}/ignore".format(my_incident_id), 
    headers=auth_headers, 
    data = {"ignore_reason" : "low_risk"}
    )

print(json.dumps(response.json(), indent=4))


### Scenario 2. I need to get more information from the committing author involved

You get more information from the developer involved by enabling sharing and sending the generated [sharing URL](https://docs.gitguardian.com/internal-repositories-monitoring/remediate/remediate-incidents#collaborating-with-developers) to the developer involved. 
*Remeber to treat these URLs as you would any other secret. The generated page will expose the original secret involved in the incident.*

When requesting the share URL you have some options: 
- auto_healing - Default: false - Allow the developer to resolve or ignore through the share link
- feedback_collection	- Default: true - Allow the developer to submit their feedback through the share link
- lifespan - Default: 0 - Lifespan, in hours, of the share link. If 0 or unset, a default value will be applied based on the workspace settings.

For this example, leave `feedback_collection` and `lifespan` as their defaults and set `auto_healing` to `true`


In [ ]:
response = requests.post(
    "https://api.gitguardian.com/v1/incidents/secrets/{}/share".format(my_incident_id), 
    headers=auth_headers, 
    data = {"auto_healing" : "true"}
    )

print(json.dumps(response.json(), indent=4))



## Congratulations 

You have now used the API to :

- List incidents
- Find a workspace member's ID and emails
- Assign an incidnet
- Drill in see all the incident information, including the occurance data
- Resolved the incident in 2 different ways

I encourage you to experiment with available filters and leverage the full [GitGuardian API reference documentation](https://api.gitguardian.com/docs) to see what other options are available. 


## Scanning for secrets with the API

There is one more API call that I want to demonstrate: scanning for secrets.

Most of the time, when you are scanning a repo, or any document, you are going to be better off using ggshield, the CLI. This is because it includes some logic to handle batching API calls better and deal with entire repositories at once. 

But in the interest of demonstration, I am including it here, scanning the first document that I told you to build when we started, `authorization.py`


In [ ]:
import os
f = open("authorization.py", "r", encoding="utf-8")
book = f.read()
response = requests.post(
    "https://api.gitguardian.com/v1/scan", 
    headers=auth_headers, 
    data = {"filename":"authorization.py", "document": book}
    )

print(json.dumps(response.json(), indent=4))

f.close()


Again, make sure you do not add this file to the git repository without accounting for it with .gitignore

Also, in this repository, you will find another notebook with some sample automations using the API. Those are all built using what we covered in this notebook. 
